In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install wget

In [ ]:
!pip install split-folders tqdm

voc 2012 dataset

In [ ]:
import wget
import os
import tarfile

%cd "/content/drive/MyDrive/project/voc"

wget.download('http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar')#test set down

fname = '/content/drive/MyDrive/project/voc/VOCtrainval_11-May-2012.tar'  
ap = tarfile.open(fname)

ap.extractall('/content/drive/MyDrive/project/voc/image')
 
ap.close()

os.remove('/content/drive/MyDrive/project/coco/image/annotations_trainval2017.zip')#tar file 삭제

import

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision.models as models
import torch.optim as optim
from torchvision import models

import numpy as np
import time
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

gpu 사용 설정

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

data preprocessing

In [ ]:
num_classes = 21
data_root = '/content/drive/MyDrive/project/voc/image/VOCdevkit/VOC2012'

In [ ]:
voc_classes = ('background', 'aeroplane', 'bicyle', 'bird', 'boat', 'bottle',
               'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor')

In [ ]:
len(voc_classes)

In [ ]:
class VOC2012dataset(torch.utils.data.Dataset):
    
    def __init__(self, num_classes, list_file, img_dir, segmentation_dir, transform=None):
        self.num_classes = num_classes
        self.images = open(list_file, "rt").read().split("\n")[:-1]
        self.transform = transform
        self.img_extension = ".jpg"
        self.segmentation_extension = ".png"
        self.image_root_dir = img_dir
        self.segmentation_root_dir = segmentation_dir

    def __len__(self):
        return len(self.images)

    
    def __getitem__(self, index):
        name = self.images[index]
        image_path = os.path.join(self.image_root_dir, name + self.img_extension)
        segmentation_path = os.path.join(self.segmentation_root_dir, name + self.segmentation_extension)

        image = self.load_image(path=image_path)
        gt_segmentation = self.load_segmentation(path=segmentation_path)

        return torch.FloatTensor(image), torch.LongTensor(gt_segmentation)

    
    def load_image(self, path=None):
        raw_image = PIL.Image.open(path)
        raw_image = np.transpose(raw_image.resize((224, 224)), (2,1,0))
        imx_t = np.array(raw_image, dtype=np.float32)/255.0
        return imx_t

    
    def load_segmentation(self, path=None):
        raw_image = PIL.Image.open(path)
        raw_image = raw_image.resize((224, 224))
        imx_t = np.array(raw_image)
        imx_t[imx_t==255] = self.num_classes-1
        return imx_t

hyperparameter

In [ ]:
batch_size = 16
num_epochs = 45
learning_rate = 0.0003

train & validation dataset, dataloader

In [ ]:
train_path = os.path.join(data_root, '/content/drive/MyDrive/project/voc/image/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt')
val_path = os.path.join(data_root, '/content/drive/MyDrive/project/voc/image/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt')

img_dir = os.path.join(data_root, "JPEGImages")
segmentation_dir = os.path.join(data_root, "SegmentationClass")

In [ ]:
train_dataset = VOC2012dataset(num_classes = num_classes, 
                                 list_file = train_path,
                                 img_dir = img_dir, 
                                 segmentation_dir=segmentation_dir)

val_dataset = VOC2012dataset(num_classes = num_classes, 
                               list_file=val_path,
                               img_dir=img_dir, 
                               segmentation_dir=segmentation_dir)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size = batch_size,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(val_dataset, 
                                         batch_size = batch_size,
                                         shuffle=False)

In [ ]:
test_path = os.path.join(data_root, 'ImageSets/Segmentation/test.txt')

test_dataset = VOC2012dataset(num_classes = num_classes, 
                               list_file=test_path,
                               img_dir=img_dir, 
                               segmentation_dir=segmentation_dir)

test_loader = torch.utils.data.DataLoader(val_dataset, 
                                         batch_size = batch_size,
                                         shuffle=False)

In [ ]:
print('Train Dataset:')
print('length :', len(train_dataset))
print('Validation Dataset:')
print('length :', len(val_dataset))

In [ ]:
print('Test Dataset:')
print('length :', len(test_dataset))

FCN 8s

In [ ]:
class FCN8s(nn.Module):
    def __init__(self, num_classes=21):
        super(FCN8s, self).__init__()
        self.relu    = nn.ReLU(inplace=True)

        # conv1 
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True) 

        # conv2 
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True) 

        # conv3
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True) 

        # conv4
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True) 

        # conv3
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True) 

        self.score_pool3 = nn.Conv2d(256, num_classes, 1)
        self.score_pool4 = nn.Conv2d(512, num_classes, 1)

        # fc1
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc2
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()

        # fc3
        self.score_fr = nn.Conv2d(4096, num_classes, kernel_size = 1)
        self.upscore2 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size = 4, stride = 2)
        self.upscore2_pool4 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size = 4, stride = 2)
        self.upscore8 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size = 16, stride = 8)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.xavier_uniform_(m.weight)

                if m.bias is not None:
                    torch.nn.init.zeros_(m.bias)


    def forward(self, x):
        h = self.relu1_1(self.conv1_1(x))
        h = self.relu1_2(self.conv1_2(h))
        h = self.pool1(h)

        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        h = self.pool2(h)

        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        pool3 = h = self.pool3(h)

        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        pool4 = h = self.pool4(h)

        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)

        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)

        pool3 = self.score_pool3(pool3)
        pool4 = self.score_pool4(pool4)

        h = self.score_fr(h)
        upscore2 = self.upscore2(h)         

        dh, dw = (pool4.size()[2] - upscore2.size()[2])//2, (pool4.size()[3] - upscore2.size()[3])//2
        upscore2_pool4 = self.upscore2_pool4(upscore2 + pool4[:, :, dh:(dh + upscore2.size()[2]), dw:(dw + upscore2.size()[3])])

        dh, dw = (pool3.size()[2] - upscore2_pool4.size()[2])//2, (pool3.size()[3] - upscore2_pool4.size()[3])//2
        upscore8 = self.upscore8(upscore2_pool4 + pool3[:, :, dh:(dh + upscore2_pool4.size()[2]), dw:(dw + upscore2_pool4.size()[3])])

        dh, dw = (upscore8.size()[2] - x.size()[2])//2, (upscore8.size()[3] - x.size()[3])//2
        return torch.sigmoid(upscore8[:, :, dh:(dh + x.size()[2]), dw:(dw + x.size()[3])])

FCN2s

In [ ]:
class FCN2s(nn.Module):
  def __init__(self, num_classes=21):
    super(FCN2s, self).__init__()
    self.relu = nn.ReLU(inplace=True)

    #convolution1
    self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
    self.relu1_1 = nn.ReLU(inplace=True)
    self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
    self.relu1_2 = nn.ReLU(inplace=True)
    self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

    #convolution2
    self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
    self.relu2_1 = nn.ReLU(inplace=True)
    self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
    self.relu2_2 = nn.ReLU(inplace=True)
    self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

    #convolution3
    self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
    self.relu3_1 = nn.ReLU(inplace=True)
    self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
    self.relu3_2 = nn.ReLU(inplace=True)
    self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
    self.relu3_3 = nn.ReLU(inplace=True)
    self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

    #convolution4
    self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
    self.relu4_1 = nn.ReLU(inplace=True)
    self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
    self.relu4_2 = nn.ReLU(inplace=True)
    self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
    self.relu4_3 = nn.ReLU(inplace=True)
    self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

    #convolution5
    self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
    self.relu5_1 = nn.ReLU(inplace=True)
    self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
    self.relu5_2 = nn.ReLU(inplace=True)
    self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
    self.relu5_3 = nn.ReLU(inplace=True)
    self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

    self.score_pool1 = nn.Conv2d(64, num_classes, 1)
    self.score_pool2 = nn.Conv2d(128, num_classes, 1)
    self.score_pool3 = nn.Conv2d(256, num_classes, 1)
    self.score_pool4 = nn.Conv2d(512, num_classes, 1)

    #fully convolution6
    self.fc6 = nn.Conv2d(512, 4096, 7)
    self.relu6 = nn.ReLU(inplace=True)
    self.drop6 = nn.Dropout2d()

    #fully convolution7
    self.fc7 = nn.Conv2d(4096, 4096, 1)
    self.relu7 = nn.ReLU(inplace=True)
    self.drop7 = nn.Dropout2d()

    #score
    self.score_fr = nn.Conv2d(4096, num_classes, kernel_size=1)
    self.upscore2_1 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2)
    self.upscore2_pool4 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2)
    self.upscore2_pool3 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2)
    self.upscore2_pool2 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2)
    self.upscore2_2 = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2)

    self._initialize_weights()

  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)

        if m.bias is not None:
          torch.nn.init.zeros_(m.bias)

  def forward(self, x):
        h = self.relu1_1(self.conv1_1(x))
        h = self.relu1_2(self.conv1_2(h))
        pool1 = h = self.pool1(h)

        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        pool2 = h = self.pool2(h)

        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        pool3 = h = self.pool3(h)

        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        pool4 = h = self.pool4(h)
        
        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)
        
        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)
        
        pool1 = self.score_pool1(pool1)
        pool2 = self.score_pool2(pool2)
        pool3 = self.score_pool3(pool3)
        pool4 = self.score_pool4(pool4)
        
        h = self.score_fr(h)
        upscore2_1 = self.upscore2_1(h)         
        
        dh, dw = (pool4.size()[2] - upscore2_1.size()[2])//2, (pool4.size()[3] - upscore2_1.size()[3])//2
        upscore2_pool4 = self.upscore2_pool4(upscore2_1 + pool4[:, :, dh:(dh + upscore2_1.size()[2]), dw:(dw + upscore2_1.size()[3])])
        
        dh, dw = (pool3.size()[2] - upscore2_pool4.size()[2])//2, (pool3.size()[3] - upscore2_pool4.size()[3])//2
        upscore2_pool3 = self.upscore2_pool3(upscore2_pool4 + pool3[:, :, dh:(dh + upscore2_pool4.size()[2]), dw:(dw + upscore2_pool4.size()[3])])

        dh, dw = (pool2.size()[2] - upscore2_pool3.size()[2])//2, (pool2.size()[3] - upscore2_pool3.size()[3])//2
        upscore2_pool2 = self.upscore2_pool2(upscore2_pool3 + pool2[:, :, dh:(dh + upscore2_pool3.size()[2]), dw:(dw + upscore2_pool3.size()[3])])

        dh, dw = (pool1.size()[2] - upscore2_pool2.size()[2])//2, (pool1.size()[3] - upscore2_pool2.size()[3])//2
        upscore2_2 = self.upscore2_2(upscore2_pool2 + pool1[:, :, dh:(dh + upscore2_pool2.size()[2]), dw:(dw + upscore2_pool2.size()[3])])

        dh, dw = (upscore2_2.size()[2] - x.size()[2])//2, (upscore2_2.size()[3] - x.size()[3])//2
        return torch.sigmoid(upscore2_2[:, :, dh:(dh + x.size()[2]), dw:(dw + x.size()[3])])

TransConv

In [ ]:
class TransConv(nn.Module):
    def __init__(self, num_classes=21):
        super(TransConv, self).__init__()
        self.relu = nn.ReLU(inplace=True)

        def Conv(in_channels, out_channels, kernel_size=3, stride=1, padding=1):

            return nn.Sequential(
                nn.Conv2d(in_channels=in_channels, 
                          out_channels=out_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding),
                nn.BatchNorm2d(out_channels),
                nn.ReLU())

        def TConv(in_channels, out_channels, kernel_size=4, stride=2, padding=1):

            return nn.Sequential(
                nn.ConvTranspose2d(in_channels=in_channels, 
                                   out_channels=out_channels,
                                   kernel_size=kernel_size, 
                                   stride=stride,
                                   padding=padding),
                nn.BatchNorm2d(out_channels),
                nn.ReLU())        

        #convolution1
        self.conv1_1 = Conv(3, 64, 3, 1, 1)
        self.conv1_2 = Conv(64, 64, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True, return_indices=True) # 1/2

        #convolution2 
        self.conv2_1 = Conv(64, 128, 3, 1, 1)
        self.conv2_2 = Conv(128, 128, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True, return_indices=True) # 1/4

        #convolution3
        self.conv3_1 = Conv(128, 256, 3, 1, 1)
        self.conv3_2 = Conv(256, 256, 3, 1, 1)
        self.conv3_3 = Conv(256, 256, 3, 1, 1)        
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True, return_indices=True) # 1/8

        #convolution4
        self.conv4_1 = Conv(256, 512, 3, 1, 1)
        self.conv4_2 = Conv(512, 512, 3, 1, 1)
        self.conv4_3 = Conv(512, 512, 3, 1, 1)        
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True, return_indices=True) # 1/16

        #convolution5
        self.conv5_1 = Conv(512, 512, 3, 1, 1)
        self.conv5_2 = Conv(512, 512, 3, 1, 1)
        self.conv5_3 = Conv(512, 512, 3, 1, 1)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True, return_indices=True)

        #fully convolution6
        self.fc6 = Conv(512, 4096, 7, 1, 0)
        self.drop6 = nn.Dropout2d(0.5)

        #fully convolution7
        self.fc7 = Conv(4096, 4096, 1, 1, 0)
        self.drop7 = nn.Dropout2d(0.5)

        #transpose convolution6
        self.fc6_tconv = TConv(4096, 512, 7, 1, 0)

        #transpose convolution5
        self.unpool5 = nn.MaxUnpool2d(2, stride=2) #7*7 -> 14*14
        self.tconv5_1 = TConv(512, 512, 3, 1, 1)
        self.tconv5_2 = TConv(512, 512, 3, 1, 1)
        self.tconv5_3 = TConv(512, 512, 3, 1, 1)

        #transpose convolution4
        self.unpool4 = nn.MaxUnpool2d(2, stride=2) #14*14 -> 28*28
        self.tconv4_1 = TConv(512, 512, 3, 1, 1)
        self.tconv4_2 = TConv(512, 512, 3, 1, 1)
        self.tconv4_3 = TConv(512, 256, 3, 1, 1)

        #transpose convolution3
        self.unpool3 = nn.MaxUnpool2d(2, stride=2) #28*28 -> 56*56
        self.tconv3_1 = TConv(256, 256, 3, 1, 1)
        self.tconv3_2 = TConv(256, 256, 3, 1, 1)
        self.tconv3_3 = TConv(256, 128, 3, 1, 1)

        # Score
        self.upscore8 = TConv(128, num_classes, 8, 4, 2)

    def forward(self, x):

        h = self.conv1_1(x) #kernel_size=3, stride=1, padding=1 -> 224*224
        h = self.conv1_2(h) #kernel_size=3, stride=1, padding=1 -> 224*224
        h, pool1_indices = self.pool1(h) #kernel_size=2, stride=2, padding=0 -> 112*112

        h = self.conv2_1(h) #kernel_size=3, stride=1, padding=1 -> 112*112
        h = self.conv2_2(h) #kernel_size=3, stride=1, padding=1 -> 112*112
        h, pool2_indices = self.pool2(h) #kernel_size=2, stride=2, padding=0 -> 56*56

        h = self.conv3_1(h) #kernel_size=3, stride=1, padding=1 -> 56*56
        h = self.conv3_2(h) #kernel_size=3, stride=1, padding=1 -> 56*56
        h = self.conv3_3(h) #kernel_size=3, stride=1, padding=1 -> 56*56   
        h, pool3_indices = self.pool3(h) #kernel_size=2, stride=2, padding=0 -> 28*28

        h = self.conv4_1(h) #kernel_size=3, stride=1, padding=1 -> 28*28
        h = self.conv4_2(h) #kernel_size=3, stride=1, padding=1 -> 28*28
        h = self.conv4_3(h) #kernel_size=3, stride=1, padding=1 -> 28*28
        h, pool4_indices = self.pool4(h) #kernel_size=2, stride=2, padding=0 -> 14*14

        h = self.conv5_1(h) #kernel_size=3, stride=1, padding=1 -> 14*14
        h = self.conv5_2(h) #kernel_size=3, stride=1, padding=1 -> 14*14
        h = self.conv5_3(h) #kernel_size=3, stride=1, padding=1 -> 14*14     
        h, pool5_indices = self.pool5(h) #kernel_size=2, stride=2, padding=0 -> 7*7

        h = self.fc6(h)
        h = self.drop6(h)

        h = self.fc7(h)
        h = self.drop7(h)

        h = self.fc6_tconv(h)

        h = self.unpool5(h, pool5_indices) #7*7 -> 14*14
        h = self.tconv5_1(h) #14*14 -> 28*28
        h = self.tconv5_2(h)
        h = self.tconv5_3(h)

        h = self.unpool4(h, pool4_indices) #14*14
        h = self.tconv4_1(h) #28*28
        h = self.tconv4_2(h)
        h = self.tconv4_3(h)

        h = self.unpool3(h, pool3_indices) #28*28
        h = self.tconv3_1(h) #56*56
        h = self.tconv3_2(h)
        h = self.tconv3_3(h)

        h = self.upscore8(h)           
        return h

loss function, optimizer

In [ ]:
import torch.nn.functional as F

def criterion(input, target, weight=None, size_average=True):
    n, c, h, w = input.size()
    nt, ht, wt = target.size()

    if h != ht and w != wt:
        input = F.interpolate(input, size=(ht, wt), mode="bilinear", align_corners=True)

    input = input.transpose(1, 2).transpose(2, 3).contiguous().view(-1, c)
    target = torch.cuda.LongTensor(target.view(-1))

    loss = F.cross_entropy(
        input, target, weight=weight, size_average=size_average, ignore_index=250
    )
    return loss

model save

In [ ]:
def save_func(model, saved_dir, file_name):
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(model, output_path)

fcn 8s

In [ ]:
torch.manual_seed(1007) 

fcn = FCN8s(num_classes=21)
fcn = fcn.to(device)
optimizer = torch.optim.SGD(params = fcn.parameters(), lr = learning_rate, weight_decay=1e-6)   

saved_fcn = '/content/drive/MyDrive/project/model/FCN8s'

fcn2s

In [ ]:
torch.manual_seed(1007) 

fcn = FCN2s(num_classes=21)
fcn = fcn.to(device)
optimizer = torch.optim.SGD(params = fcn.parameters(), lr = learning_rate, weight_decay=1e-6)   

saved_fcn = '/content/drive/MyDrive/project/model/FCN2s'

transconv

In [ ]:
torch.manual_seed(1007) 

tc = TransConv(num_classes=21)
tc = tc.to(device)
optimizer = torch.optim.SGD(params = tc.parameters(), lr = learning_rate, weight_decay=1e-6)   

saved_tconv = '/content/drive/MyDrive/project/model/TransConv'

train & validation function

In [ ]:
def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, file_name, device):
    print('Start train')
    best_loss = 9999999
    for epoch in range(num_epochs):
        for step, (image, segmentation) in enumerate(data_loader):
            image = image.type(torch.float32)
            segmentation = segmentation.type(torch.long)
            image, segmentation = image.to(device), segmentation.to(device)
            
            outputs = model(image) 
            loss = criterion(outputs, segmentation)
            
            optimizer.zero_grad()          
            loss.backward()
            optimizer.step()
            
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_loader), loss.item()))
                
        if (epoch + 1) % 1 == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_func(model, saved_dir, file_name)

In [ ]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch))
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        correct = 0
        
        for step, (image, segmentation) in enumerate(data_loader):
            image = image.type(torch.float32)
            segmentation = segmentation.type(torch.long)
            image, segmentation = image.to(device), segmentation.to(device)

            outputs = model(image)
            prediction = torch.argmax(outputs, 1)
            loss = criterion(outputs, segmentation)

            correct += prediction.eq(segmentation).sum().item()
            total_loss += loss
            cnt += 1

        avrg_loss = total_loss / cnt
        accuracy = correct / cnt
        print('Validation #{}  Average Loss: {:.4f} Accuracy : {:.4f}'.format(epoch, avrg_loss, accuracy))
   
    model.train()
    return avrg_loss

modle train - fcn 8s

In [ ]:
train(num_epochs, fcn, train_loader, criterion, optimizer, saved_fcn, 'FCN8s_model.pt', device)

model train - fcn 2s

In [ ]:
train(num_epochs, fcn, train_loader, criterion, optimizer, saved_fcn, 'FCN2s_model.pt', device)

model train - transconv

In [ ]:
train(num_epochs, tc, train_loader, criterion, optimizer, saved_tconv, 'TransConv_model.pt', device)

load train model - fcn 2s

In [ ]:
fcn = torch.load(os.path.join(saved_fcn, 'FCN2s_model.pt'))

load train model - transconv

In [ ]:
fcn = torch.load(os.path.join(saved_tconv, 'TransConv_model.pt'))